In [ ]:

path_list = ["11112024_BDMetaData_CanRuti.xlsx", "11112024_BDMetaData_santpau.xlsx", "28102024_BDMetaData_delmar.xlsx", "28102024_BDMetaData_mutuaterrassa.xlsx"]




In [ ]:
import os
from typing import Optional, List, Dict, Tuple, Union

import pandas as pd
from datetime import datetime

from collections import Counter


# TODO: add other tables

class RadioLungDataset():
    
    def __init__(self, path_df: str, mode = None):
        
        self.df = self.merge_datasets(path_df)
        self.dataset_name = "RadioLung"
        self.map_dataset()
        self.input_info = {} # com estan cada feature_key
        self.map_feature_token_space()
        self.mode = None

    def map_dataset(self):
        
        self.df.dropna(inplace=True)

        self.df["birthdate"] = pd.to_datetime(self.df["birthdate"])
        self.df["age"] = datetime.today().year - self.df["birthdate"].dt.year

        self.df.drop(columns=["birthdate"], inplace=True)
        self.df["type"] = self.df["type"].map({"Malignant": 1, "Benign": 0})

    def prova(self):
        if self.mode != None:
            print(self._df)


    def __len__(self):
        return self.df.shape[0]
    

    def get_df(self):
        return self.df
    
    def map_feature_token_space(self) -> None:
        feature_keys = self.df.columns
        self.map_token_space = {}
        for feature_key in feature_keys:
           
            token_space = self.df[feature_key].unique().tolist() #token_space = self.df[feature_key].unique().flatten().tolist()
            if pd.api.types.is_string_dtype(self.df[feature_key] ) == True:
                self.input_info[feature_key] = {"type": str, "dim": 2} 
            else:
                self.input_info[feature_key] = {"type": int, "dim": 2, 'len':1}# ha petat i tb i he ficat len 
             
                self.df[feature_key]= (self.df[feature_key].apply(lambda x: [x])) # he ficat tots es num dins una llista pq vol aquest format
            
           #todo: modificar pq puguis ficar aquesta linia enlloc de int/float -->  (type(self.df[feature_key].iloc[0]).__name__)


    def case1_case2_binary(self): #s'han de natejar dades crec
        feature_bin = ["air_pollution", "ph_cancer", "COPD", "ASTHMA", "bronchiectasis", "TB", "pneumonia"]
        
        dic_bi = {}
        for i, feature in enumerate(feature_bin):
            dic_bi[feature] = {0: i * 2, 1: i * 2 + 1}
        
        for feature in feature_bin:
            self.df[feature] = self.df[feature].map(dic_bi[feature])
        return dic_bi
        

    def merge_datasets(self, list_paths):
        df_list = []  
        selected_columns = ["patient_id", "hospital", "type", "birthdate", "sex", "BMI", "education",
                            "air_pollution", "smoking", "packyear_index", "fh_cancer",
                            "ph_cancer", "COPD", "FVC", "indice"]

        for p in list_paths:
            data_read = pd.read_excel(p, header=0)
            data_read = data_read.reindex(columns=selected_columns)
            df_list.append(data_read)

        self.df = pd.concat(df_list, ignore_index=True)
        return self.df
         


    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        sample_return = {fkey: [value] if fkey != "type" else value for fkey, value in sample.items()}
        sample_return.update({"patient_id": sample_return["patient_id"]})
      #  sample_return.pop("id")
        return sample_return



In [18]:

dataset = RadioLungDataset(path_df=path_list)


     patient_id hospital       type  birthdate    sex   BMI  \
0             1  canruti  Malignant 1945-07-22  Woman  33.0   
1             8  canruti  Malignant 1942-01-08    Man  33.0   
2             9  CanRuti  Malignant 1949-05-14    Man  27.0   
3            10  CanRuti  Malignant 1943-06-07    Man  31.2   
4            11  CanRuti  Malignant 1943-06-27    Man  23.5   
..          ...      ...        ...        ...    ...   ...   
105         204  CanRuti     Benign 1963-06-01    Man  37.0   
106         232  CanRuti     Benign 1949-02-08  Woman   0.0   
107         233  CanRuti     Benign 1943-09-05    Man   0.0   
108         234  CanRuti     Benign 1957-03-11    Man  28.9   
109         235  CanRuti  Malignant 1956-02-19    Man  22.9   

                      education  air_pollution  smoking  packyear_index  \
0     Post high school training              1  Current            12.0   
1             High scrool grade              0   Former            40.0   
2             High